#### Construction de la base d'apprentissage

In [1]:
# Pour construire la base d'apprentissage seulement
#from Bio import SeqIO
#import RNA

In [2]:
import numpy as np
import pandas as pd
import csv
from random import shuffle
from Bio import SeqIO
import RNA

In [3]:
def transformer_sequence(sequence,long):
    nucleotides = np.zeros((4,int(max(long,25))))
    cnt = 0
    for lettre in sequence:
        if lettre=='a':
            nucleotides[0,cnt] = 1
            cnt = cnt+1
        elif lettre=='c':
            nucleotides[1,cnt] = 1
            cnt = cnt+1
        elif lettre=='g':
            nucleotides[2,cnt] = 1
            cnt = cnt+1
        else:
            nucleotides[3,cnt] = 1
            cnt = cnt+1
    return nucleotides

In [4]:
def transformer_structure(structure):
    long = len(structure)
    a = np.zeros(25)
    a[:long] = structure
    return a

In [5]:
def traduire(seq,strand):
    nucleotides = ''
    if strand:
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'a'
            elif lettre=='C':
                nucleotides=nucleotides+'c'
            elif lettre=='G':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    else:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'u'
            elif lettre=='C':
                nucleotides=nucleotides+'g'
            elif lettre=='G':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    return nucleotides

### Génération des bdd (ne pas exécuter)

In [5]:
parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")
dict_fasta = dict([(seq.id, seq) for seq in parser])

In [20]:
bdd = pd.read_csv("rise_human_transcriptome.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)

# Ne garder que les interactions miRNA - mRNA
arg_mi = np.argwhere(arrbdd[:,14]=='protein_coding')[:,0]
arg_m = np.argwhere(arrbdd[:,15]=='protein_coding')[:,0]
args = [k for k in arg_mi if k in arg_m]
args = np.array(args)
arrbdd = arrbdd[args]

#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'chrM') & (arrbdd[:,3] != 'chrM'))
arrbdd = arrbdd[arg_sansM[:,0]]
#enlever les champs vides
arg_sansM = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_sansM))
varrbdd = arrbdd[arg_sansM[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False
arg_sansM=[]
arrbdd=[]
bdd=[]

for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 30)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 30)]

varrbdd = varrbdd[:,0]

In [37]:
len(varrbdd)

22183

### Tests anciens

In [8]:
i = 0
pos = bdd[i,:4]
ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
seq = dict_fasta[ide][begin:end].format("fasta").split('\n')
pos[0] = traduire(seq[1]+seq[2],bdd[i,8])
ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
seq = dict_fasta[ide][begin:end].format("fasta").split('\n')
pos[1] = traduire(seq[1]+seq[2],bdd[i,9])
w = len(pos[0])
pos[2] = np.array(RNA.pfl_fold_up(pos[0],1,w,w))[1:,1]
pos[3] = np.array(RNA.pfl_fold_up(pos[1],1,101,101))[1:,1]

In [34]:
pos[0][31:73]

'agggggaacgcuggccucugaaacuagcucugggaccggggu'

In [32]:
pos[1][40:63]

'uccuccugcauucaaccgugcag'

In [26]:
def rev(s):
    t=''
    for i in s:
        t = i + t
    return t
rev('aacuccuccugcauucaaccgugca')

'acgugccaacuuacguccuccucaa'

### Création base

In [ ]:
bdd=varrbdd
varrbdd=[]

In [40]:
writer = csv.writer(open("genomes/positifs_m-m.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)):
    pos = bdd[i,:2] # 4 si structures secondaires
    ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
    pos[0] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,8])
    ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
    pos[1] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,9])
    #w = len(pos[0])
    #pos[2] = np.array(RNA.pfl_fold_up(pos[0],1,w,w))[1:,1]
    #w = len(pos[1])
    #pos[3] = np.array(RNA.pfl_fold_up(pos[1],1,w,w))[1:,1]
    writer.writerow(pos)        


In [43]:
bdd = pd.read_csv("rise_human_transcriptome.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)

# Ne garder que les interactions miRNA - mRNA
arg_mi = np.argwhere(arrbdd[:,14]=='protein_coding')[:,0]
arg_m = np.argwhere(arrbdd[:,15]=='protein_coding')[:,0]
args = [k for k in arg_mi if k in arg_m]
args = np.array(args)
arrbdd = arrbdd[args]

#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'chrM') & (arrbdd[:,3] != 'chrM'))
arrbdd = arrbdd[arg_sansM[:,0]]
#enlever les champs vides
arg_sansM = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
varrbdd = arrbdd[arg_sansM[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False
arg_sansM=[]
arrbdd=[]
bdd=[]

for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 30)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 30)]

varrbdd = varrbdd[:,0]

/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


37385


In [45]:
len(varrbdd)

22183

In [46]:
bdd=varrbdd
varrbdd=[]

In [47]:
writer = csv.writer(open("genomes/negatifs_m-m.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)): 
    neg = bdd[i,:2]
    ide, begin, end = [bdd[i,0],bdd[i,1]+30,bdd[i,2]+30]
    neg[0] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,8])
    ide, begin, end = [bdd[i,3], bdd[i,4]+30, bdd[i,5]+30]
    neg[1] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,9])
    #w = len(neg[0])
    #neg[2] = np.array(RNA.pfl_fold_up(neg[0],1,w,w))[1:,1]
    #neg[3] = np.array(RNA.pfl_fold_up(neg[1],1,101,101))[1:,1]
    writer.writerow(neg)

### Récupération et vérification des bases de données (exécuter à partir d'ici pour l'apprentissage)

### Réseau de neurones basé sur iDeep

In [6]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D

/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [7]:
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
#from keras.optimizers import kl_divergence
from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse

from keras.layers import Merge

/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [8]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 


In [9]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

verif = pd.read_csv("genomes/negatifs_m-m.csv", sep = "\t",header=None)
verif = np.array(verif)
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            

neg = verif

verif = pd.read_csv("genomes/positifs_m-m.csv", sep = "\t",header=None)  
verif = np.array(verif)
for i in range(len(verif)):
    for j in range(2):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            
pos = verif
verif = []
bdd = np.concatenate((pos,neg))
pos = []
neg = []
labels = np.zeros((len(bdd),1))

bdd = np.concatenate((bdd,labels),axis=1)
for i in range(int(len(bdd)/2)):
    bdd[i,2]=1
bdd = bdd[:int(len(bdd)*0.9)]
labels=[]

In [11]:
nb_train = 20000
nb_val = 500
nb_test = 1000

In [10]:
# shuffle total

indices = np.arange(len(bdd))
shuffle(indices)
bdd = bdd[indices]
indices=[]

# 
l = len(bdd)
matrice = np.zeros((l,30,30,3)) #4

for i in range(l):
    seq1 = bdd[i,0]
    seq2 = bdd[i,1]
    for j in range(len(seq1)):
        for k in range(len(seq2)):
            if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                matrice[i,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                matrice[i,j,k,1] = 1
            elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                matrice[i,j,k,2] = 1
    

In [12]:
matrice_train = matrice[:nb_train]
matrice_val = matrice[nb_train:nb_train+nb_val]
matrice_test = matrice[-nb_test:]
labels = bdd[:,2]

y = labels[:nb_train]
y = keras.utils.np_utils.to_categorical(y,2)
val_y = labels[nb_train:nb_train+nb_val]
val_y = keras.utils.np_utils.to_categorical(val_y,2)
true_y = labels[-nb_test:]
bdd = []

In [133]:
# alternative
l = len(bdd)
m = int(l/2)
matrice = np.zeros((l,30,30,3)) #4

for i in range(l):
    seq1 = bdd[i,0]
    seq2 = bdd[i,1]
    for j in range(len(seq1)):
        for k in range(len(seq2)):
            if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                matrice[i,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                matrice[i,j,k,1] = 1
            elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                matrice[i,j,k,2] = 1

prop_1 = 0.6    # proportion d'exemples positifs dans le train

ind_p = np.arange(m)
shuffle(ind_p)
indices_train = ind_p[:int(nb_train*prop_1)]
indices_val = ind_p[int(nb_train*prop_1):int(nb_train*prop_1)+int(0.5*nb_val)]
indices_test = ind_p[-int(nb_test/2):]

ind_n = np.arange(m) + m
shuffle(ind_n)
ind_train_neg = ind_n[:int(nb_train*(1-prop_1))]
ind_val_neg = ind_n[int(nb_train*(1-prop_1)):int(nb_train*(1-prop_1))+int(0.5*nb_val)]
ind_test_neg = ind_n[-int(nb_test/2)]

ind_n = []
ind_p = []

indices_train = np.concatenate((indices_train,ind_train_neg))
shuffle(indices_train)
ind_train_neg = []
indices_val = np.concatenate((indices_val,ind_val_neg))
shuffle(indices_val)
ind_val_neg = []
indices_test = np.concatenate((indices_test,ind_test_neg))
shuffle(indices_test)
ind_test_neg = []

matrice_train = matrice[indices_train]
matrice_val = matrice[indices_val]
matrice_test = matrice[indices_test]

#matrice_train = np.concatenate((matrice[:int(nb_train*0.7)],matrice[int(l/2):int(l/2)+int(nb_train*0.3)]))
#matrice_val = np.concatenate((matrice[:nb_train+int(nb_val*0.5)],matrice[int(l/2):nb_train+int(l/2)+int(nb_val*0.5)]))
#matrice_test = np.concatenate((matrice[int(l/2)-nb_test:int(l/2)],matrice[-nb_test:]))

labels = bdd[:,2]

y = labels[indices_train]
y = keras.utils.np_utils.to_categorical(y,2)
val_y = labels[indices_val]
val_y = keras.utils.np_utils.to_categorical(val_y,2)
true_y = labels[indices_test]
bdd = []     

In [13]:
matrice_train[1,1]

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [14]:
def merge_networks_train_predict():
    
    print('training', nb_train)
    
    hid = 32
    
    tableau_net =  get_cnn_network()         
    
    #y, encoder = preprocess_labels(training_label)
    #val_y, encoder = preprocess_labels(validation_label, encoder = encoder)
       
    
    model = Sequential()
    model.add(tableau_net)
    model.add(BatchNormalization())
    
    model.add(Dropout(0.5))
    print(hid)
    model.add(Dense(2, input_shape=(hid,)))
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    
    #checkpointer = ModelCheckpoint(filepath="models/bestmodel.hdf5", verbose=0, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=0)
    #validation_data=(np.transpose(validmat['validxdata'],axes=(0,2,1)), validmat['validdata']), callbacks=[checkpointer,earlystopper]
    print('model training')
    model.fit(matrice_train, y, batch_size=32, epochs=25, verbose=0, validation_data=(matrice_val, val_y), callbacks=[earlystopper])
    
    # test   
    
    print('predicting')
    
        
    predictions = model.predict_proba(matrice_test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

    
    return model

In [16]:
merge_networks_train_predict()

training 20000
configure cnn network
(None, 23, 23, 128)
(None, 6, 6, 32)
32
model training
predicting
[9.60029602e-01 6.60396814e-01 4.70581204e-01 8.54571834e-02
 5.08384883e-01 9.80807900e-01 9.38623667e-01 9.66124594e-01
 9.71181631e-01 2.67716199e-01 8.88967037e-01 9.18532848e-01
 9.71960604e-01 2.10281506e-01 3.30641359e-01 8.21014404e-01
 3.73472273e-01 8.66120934e-01 1.11460276e-01 2.13824108e-01
 2.07870185e-01 1.01633981e-01 1.04260184e-01 9.62626785e-02
 9.26189542e-01 8.54132548e-02 6.32947028e-01 2.86731124e-01
 9.64920521e-01 6.12917483e-01 9.64563370e-01 5.06604075e-01
 3.84419590e-01 8.21527719e-01 1.10484086e-01 6.62164927e-01
 4.70311970e-01 8.75177205e-01 6.53212011e-01 8.98903131e-01
 9.15468931e-01 7.23839775e-02 1.77150860e-01 6.47880137e-01
 1.84435453e-02 8.81997287e-01 6.79534495e-01 8.52999464e-03
 8.02689850e-01 9.93079066e-01 8.36860180e-01 7.47546554e-01
 8.83082271e-01 8.08321059e-01 2.68528521e-01 5.57460546e-01
 8.19684267e-01 9.70277727e-01 7.46810138e-

In [15]:
def get_cnn_network():    
    
    print('configure cnn network')
    nbfilter = 128

    model = Sequential()
    model.add(Conv2D(filters = nbfilter, kernel_size=(8, 8), padding='valid', input_shape=(30,30,3),strides=(1,1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    print(model.output_shape)
    model.add(Conv2D(filters = 64, kernel_size = (6, 6),padding='valid',input_shape=(23,23,128)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(filters = 32, kernel_size = (4, 4),padding='valid',input_shape=(20,20,64)))
    model.add(Activation('relu'))
    print(model.output_shape)
    model.add(Flatten())
        
    
    return model

### ARCHIVES

### Récupération rapide des séquences à partir de chr_nb start end

In [ ]:
# download genome at ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_human/release_24/GRCh38.primary_assembly.genome.fa.gz

parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")

dict_fasta = dict([(seq.id, seq) for seq in parser])

ide, begin, end = ['chr17',8222876,8222895]

dict_fasta[ide][begin:end].format("fasta").split('\n')[1]

### A partir de là c'est l'ancienne méthode (à conserver au cas où, archive)

#### On  sauvegarde la base de données dans un fichier csv

In [28]:
browser = webdriver.Chrome()
import csv

writer = csv.writer(open("genomes/database.csv", 'w',newline=''),delimiter="\t")
for i,el in enumerate(varrbdd):
    row = transformer_database2(varrbdd[i])    
    writer.writerow(row)

On cherche les trous dans les données

In [7]:
bdd_5000_trouee = pd.read_csv("genomes/database.csv", sep = "\t",header = None)

In [8]:
bdd_5000_trouee = np.array(bdd_5000_trouee)
l = []
for i,el in enumerate(bdd_5000_trouee):
    if type(bdd_5000_trouee[i,0])==float or type(bdd_5000_trouee[i,1])==float:
        a=0
    elif len(bdd_5000_trouee[i,0])==0 or len(bdd_5000_trouee[i,0])==0:
        a=0
    else:
        l.append(i)
l = np.array(l)
len(l)

4983

In [23]:
bdd_miRNA_mRNA = bdd_5000_trouee[l]

In [26]:
bdd_miRNA_mRNA = nettoyer(bdd_miRNA_mRNA)

In [29]:
bdd_miRNA_mRNA = traduire_séquences(bdd_miRNA_mRNA)

In [20]:
bdd_apprentissage = construire_seq_et_structure_bdd(bdd_miRNA_mRNA[:15])

#### A partir de mainenant, on essaye d'obtenir les probabilités d'accesibilité de chaque nucléotide sur une séquence quelconque ARN par exemple 'ACGUUGUCACACGAU'

In [10]:
def traduire_test(seq,strand):
    nucleotides = ''
    if strand:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='a':
                nucleotides=nucleotides+'u'
            elif lettre=='c':
                nucleotides=nucleotides+'g'
            elif lettre=='g':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    else:
        for lettre in seq:
            if lettre=='a':
                nucleotides=nucleotides+'a'
            elif lettre=='c':
                nucleotides=nucleotides+'c'
            elif lettre=='g':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    return nucleotides

In [55]:
traduire_test('gaagacactcttgtctcagtaaaaggtaaaggagggctcgtccgggatttgaacccgggacctctcgcacccgaagcgagaatcatacccctagaccaacgagccgacgtgcggacgttgccgcgaaccgccttagaggtcgtgccaggcttgctgtagtg',True)

'cacuacagcaagccuggcacgaccucuaaggcgguucgcggcaacguccgcacgucggcucguuggucuagggguaugauucucgcuucgggugcgagaggucccggguucaaaucccggacgagcccuccuuuaccuuuuacugagacaagagugucuuc'

In [ ]:
'gaagacactcttgtctcagtaaaaggtaaaggagggctcgtccgggatttgaacccgggacctctcgcacccgaagcgagaatcatacccctagaccaacgagccgacgtgcggacgttgccgcgaaccgccttagaggtcgtgccaggcttgctgtagtg'

In [12]:
bdd = pd.read_csv("rise_human_transcriptome (copie).csv", sep = "\t",header=None)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
# bdd convertie panda -> numpy array et seuls les 70 000 premiers RRIs retenus
arrbdd = np.array(bdd)[:70000]

#enlever 'chr' et formater les numéros de chromosomes
for i,el in enumerate(arrbdd[:,0]):
    arrbdd[i,0] = arrbdd[i,0][3:]

for i,el in enumerate(arrbdd[:,0]):
    if arrbdd[i,0] == 'X':
        arrbdd[i,0] = '23'
    if arrbdd[i,0] == 'Y':
        arrbdd[i,0] = '24'
    if arrbdd[i,0] == '1':
        arrbdd[i,0] = '01'
    if arrbdd[i,0] == '2':
        arrbdd[i,0] = '02'
    if arrbdd[i,0] == '3':
        arrbdd[i,0] = '03'
    if arrbdd[i,0] == '4':
        arrbdd[i,0] = '04'
    if arrbdd[i,0] == '5':
        arrbdd[i,0] = '05'
    if arrbdd[i,0] == '6':
        arrbdd[i,0] = '06'
    if arrbdd[i,0] == '7':
        arrbdd[i,0] = '07'
    if arrbdd[i,0] == '8':
        arrbdd[i,0] = '08'
    if arrbdd[i,0] == '9':
        arrbdd[i,0] = '09'

for i,el in enumerate(arrbdd[:,3]):
    arrbdd[i,3] = arrbdd[i,3][3:]

for i,el in enumerate(arrbdd[:,3]):
    if arrbdd[i,3] == 'X':
        arrbdd[i,3] = '23'
    elif arrbdd[i,3] == 'Y':
        arrbdd[i,3] = '24'  
    elif arrbdd[i,3] == '1':
        arrbdd[i,3] = '01'
    elif arrbdd[i,3] == '2':
        arrbdd[i,3] = '02'
    elif arrbdd[i,3] == '3':
        arrbdd[i,3] = '03'
    elif arrbdd[i,3] == '4':
        arrbdd[i,3] = '04'
    elif arrbdd[i,3] == '5':
        arrbdd[i,3] = '05'
    elif arrbdd[i,3] == '6':
        arrbdd[i,3] = '06'
    elif arrbdd[i,3] == '7':
        arrbdd[i,3] = '07'
    elif arrbdd[i,3] == '8':
        arrbdd[i,3] = '08'
    elif arrbdd[i,3] == '9':
        arrbdd[i,3] = '09'


#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'M') & (arrbdd[:,3] != 'M'))
varrbdd = arrbdd[arg_sansM[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = 'true'
    else:
        varrbdd[i,8] = 'false'

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = 'true'
    else:
        varrbdd[i,9] = 'false'


#int str
for i,el in enumerate(varrbdd[:,1]):
    if type(el) == int:
        varrbdd[i,1] = str(el)
for i,el in enumerate(varrbdd[:,2]):
    if type(el) == int:
        varrbdd[i,2] = str(el)
for i,el in enumerate(varrbdd[:,4]):
    if type(el) == int:
        varrbdd[i,4] = str(el)
for i,el in enumerate(varrbdd[:,5]):
    if type(el) == int:
        varrbdd[i,5] = str(el)

#génération position début et fin de séquence (nucléotides)

for i,el in enumerate(varrbdd[:,1]):
    milieu = int((int(varrbdd[i,1])+int(varrbdd[i,2]))/2)
    varrbdd[i,1] = str(milieu-50)
    varrbdd[i,2] = str(milieu+50)
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = str(milieu-50)
    varrbdd[i,5] = str(milieu+50)

# Génération URLs

#for i,el in enumerate(varrbdd[:,6]):
 #   varrbdd[i,6] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,0]+"?report=genbank&from="+varrbdd[i,1]+"&to="+varrbdd[i,2]+"&strand="+varrbdd[i,8]+".html"
#for i,el in enumerate(varrbdd[:,6]):
 #   varrbdd[i,6] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,0]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,1]+'&to='+varrbdd[i,2]

#for i,el in enumerate(varrbdd[:,7]):
 #   varrbdd[i,7] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,3]+"?report=genbank&from="+varrbdd[i,4]+"&to="+varrbdd[i,5]+"&strand="+varrbdd[i,9]+".html"
#for i,el in enumerate(varrbdd[:,7]):
 #   varrbdd[i,7] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,3]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,4]+'&to='+varrbdd[i,5]

In [ ]:
def recuperer_sequence(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(4)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    browser.quit()
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("span",{'class':"ff_line"})
    rep = seq[0].text+seq[1].text+seq[2].text+seq[3].text
    rep = ''.join(rep.split())
    return rep

def recuperer_sequence2(url):
    browser.get(url)
    time.sleep(1.3)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("div",{'class':"seq gbff"})
    rep = seq[0].text   
    return rep

def transformer_database(bddrow):
    sequence=recuperer_sequence(bddrow[6])
    sequence=transformer_sequence(sequence,len(sequence))
    sequencebis=recuperer_sequence(bddrow[7])
    sequencebis=transformer_sequence(sequencebis,len(sequencebis))    
    bddrow = np.array([bddrow[18],bddrow[19],bddrow[20],bddrow[21],sequence,bddrow[0],bddrow[1],bddrow[2],bddrow[8],bddrow[10],bddrow[11],bddrow[14],bddrow[16],sequencebis,bddrow[3],bddrow[4],bddrow[5],bddrow[9],bddrow[12],bddrow[13],bddrow[15],bddrow[17]], dtype=object)
    return bddrow

def transformer_database2(bddrow):
    sequence=recuperer_sequence2(bddrow[6])
    sequencebis=recuperer_sequence2(bddrow[7])
    bddrow = np.array([sequence,sequencebis,bddrow[8],bddrow[9]], dtype=object)
    return bddrow

def construire_seq_et_structure_bdd(bdd):
    for i,el in enumerate(bdd):
        bdd[i,2] = np.array(RNA.pfl_fold_up(bdd[i,0],1,201,201))[1:,1]
        bdd[i,3] = np.array(RNA.pfl_fold_up(bdd[i,1],1,201,201))[1:,1]
        #bdd[i,0] = transformer_sequence(bdd[i,0],201)
        #bdd[i,1] = transformer_sequence(bdd[i,1],201)
    return bdd

def nettoyer(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=bdd[i,0].split('\n')[1]+bdd[i,0].split('\n')[2]+bdd[i,0].split('\n')[3]
        bdd[i,1]=bdd[i,1].split('\n')[1]+bdd[i,1].split('\n')[2]+bdd[i,1].split('\n')[3]
    return bdd

def traduire_séquences(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=traduire(bdd[i,0],bdd[i,2])
        bdd[i,1]=traduire(bdd[i,1],bdd[i,3])
    return bdd